In [1]:
import cv2
from PIL import Image
import numpy as np

%matplotlib inline
from mtcnn import MTCNN
from keras.models import load_model

In [2]:
detector = MTCNN()

# load the model
gender_model = load_model('gender_model.h5')
emotion_model = load_model('emotion_model.h5')

In [3]:
def detect_face(img):
    
    mt_res = detector.detect_faces(img)
    return_res = []
    
    for face in mt_res:
        x, y, width, height = face['box']
        center = [x+(width/2), y+(height/2)]
        max_border = max(width, height)
        
        # center alignment
        left = max(int(center[0]-(max_border/2)), 0)
        right = max(int(center[0]+(max_border/2)), 0)
        top = max(int(center[1]-(max_border/2)), 0)
        bottom = max(int(center[1]+(max_border/2)), 0)
        
        # crop the face
        cimg = img[top:top+max_border, 
                           left:left+max_border, :]
        center_img = np.array(Image.fromarray(cimg).resize([128, 128]))
        
        # create predictions
        gender_pred = gender_model.predict(center_img.reshape(1,128,128,3))[0][0]
        
        # convert to grey scale then predict using the emotion model
        gray_img = np.array(Image.fromarray(cimg).resize([48, 48]))
        emotion_pred = emotion_model.predict(gray_img.reshape(1, 48, 48, 3))
        
        # output to the cv2
        return_res.append([top, right, bottom, left, gender_pred, emotion_pred])
        
    return return_res

In [9]:
# Get a reference to webcam 
video_capture = cv2.VideoCapture(0)

emotion_dict = {
    0: 'Anger',
    1: 'Contempt', 
    2: 'Disgust',
    3: 'Fear',
    4: 'Happy',
    5: 'Sadness',
    6: 'Surprise'
}

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color 
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces in the current frame of video
    face_locations = detect_face(rgb_frame)

    # Display the results
    for top, right, bottom, left, gender_pred, emotion_pred in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        
        gender_text = 'Male' if gender_pred > 0.5 else 'Female'
        cv2.putText(frame, 'Gender: {}({:.3f})'.format(gender_text, gender_pred), (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        cv2.putText(frame, 'Emotion: {}({:.3f})'.format(emotion_dict[np.argmax(emotion_pred)], np.max(emotion_pred)), (left, top-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(20) == 27:
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [16]:
from keras.models import load_model
from mtcnn import MTCNN
from PIL import Image


import numpy as np


import cv2


detector = MTCNN()
emotion_dict = {
    0: 'Anger',
    1: 'Contempt', 
    2: 'Disgust',
    3: 'Fear',
    4: 'Happy',
    5: 'Sadness',
    6: 'Surprise'
}

# load the models
gender_model = load_model('gender_model.h5')
emotion_model = load_model('emotion_model.h5')

print('Model loaded successfully')


def detect_face(img):
    
    
    mt_res = detector.detect_faces(img)
    
    for i, face in enumerate(mt_res):
        print(face, i)
        
        x, y, width, height = face['box']
        center = [x+(width/2), y+(height/2)]
        max_border = max(width, height)
        
        # center alignment
        left = max(int(center[0]-(max_border/2)), 0)
        right = max(int(center[0]+(max_border/2)), 0)
        top = max(int(center[1]-(max_border/2)), 0)
        bottom = max(int(center[1]+(max_border/2)), 0)
        
        # crop the face
        cimg = img[top:top+max_border, 
                           left:left+max_border, :]
        center_img = np.array(Image.fromarray(cimg).resize([128, 128]))
        
        # gender prediction
        gender_pred = gender_model.predict(center_img.reshape(1,128,128,3))[0][0]
        
        # emotion prediction
        gray_img = np.array(Image.fromarray(cimg).resize([48, 48]))
        emotion_pred = emotion_model.predict(gray_img.reshape(1, 48, 48, 3))
        
        # Draw a box around the face
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
        gender_text = 'Male' if gender_pred > 0.5 else 'Female'
        cv2.putText(img, 'Gender: {}'.format(gender_text), (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        cv2.putText(img, 'Emotion: {}'.format(emotion_dict[np.argmax(emotion_pred)]), (left, top-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        
        cv2.imwrite('predicted_image_{}.jpg'.format(i), img)


if __name__ == '__main__' :
    
    image_path = input('Enter image name with format : ')
    img = cv2.imread(image_path)
    detect_face(img) 

Model loaded successfully
Enter image name with format : mama.jpg
{'box': [131, 98, 334, 411], 'confidence': 0.9783836007118225, 'keypoints': {'left_eye': (227, 268), 'right_eye': (382, 252), 'nose': (307, 323), 'mouth_left': (245, 418), 'mouth_right': (388, 406)}} 0
